# Senitment Prediction for Robo Eye Animation

The project, involves a dynamic robot eye animation displayed on a 0.96" OLED screen that changes based on the sentiment of text input provided by the user. The hardware controller used for this project is the CAP10 Pratham, a made-in-India board. This project offers a unique way to integrate emotional intelligence into robotics, enabling bots to visually express emotions in response to user interactions.

**Author:** Asutosh Pati ([https://www.linkedin.com/in/asutoshpati/](https://www.linkedin.com/in/asutoshpati/))  
**Date:** 21-Aug-2024  
**Versions:**  
- V1.0: Initial Release
- V1.1: Change communication method from Serial to API call

### Install & Import Modules

In [ ]:
# !pip install tensorflow==2.13.1

In [1]:
import pickle
import re

import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\asuto\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asuto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Preprocess the sentence

In [3]:
stop_words = set(stopwords.words("english"))
lemmatizer= WordNetLemmatizer()

In [4]:
def lower_case(text):
    text = text.split()
    text = [y.lower() for y in text]
    return " ".join(text)

lower_case("Oh, Yes! I have ssen that.")

'oh, yes! i have ssen that.'

In [5]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

remove_urls("Have you visited https://www.google.co.in/")

'Have you visited '

In [6]:
def remove_punctuations(text):
    # Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )
    
    # remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

remove_punctuations("Are you sure; It's amazing!")

'Are you sure It s amazing'

In [7]:
def remove_numbers(text):
    text = ''.join([i for i in text if not i.isdigit()])
    return text

remove_numbers("Have you seen the prime minister's speech at 10 am")

"Have you seen the prime minister's speech at  am"

In [8]:
def remove_stop_words(text):
    text = [i for i in str(text).split() if i not in stop_words]
    return " ".join(text)

remove_stop_words("I am a very good boy; is it true")

'I good boy; true'

In [9]:
def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(y) for y in text]
    return " ".join(text)

lemmatization("I love to watch Footbal. What do you like to do?")

'I love to watch Footbal. What do you like to do?'

In [10]:
def preprocess_sentence(sentence):
    sentence = lower_case(sentence)
    sentence = remove_urls(sentence)
    sentence = remove_punctuations(sentence)
    sentence = remove_numbers(sentence)
    sentence = remove_stop_words(sentence)
    sentence = lemmatization(sentence)
    return sentence

preprocess_sentence("Arise, awake, and stop not until the goal is reached. - Swami Vivekananda")

'arise awake stop goal reached swami vivekananda'

### Load Models

In [11]:
# import label encoder created during training

le = None
with open("./model/label_encoder.pkl",'rb') as file:
    le = pickle.load(file)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
# import tokenizer created during training

tokenizer = None
with open("./model/tokenizer.json") as file:
    tokenizer_json = file.read()
    tokenizer = tokenizer_from_json(tokenizer_json)

In [13]:
# import the pre-trained model

model_path = "./model/Sentiment_analysis_Eng-V3.h5"
model = load_model(model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 229, 200)          2865000   
                                                                 
 bidirectional (Bidirection  (None, 229, 512)          935936    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 229, 256)          656384    
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 229, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_3 (Bidirecti  (None, 229, 128)          164352    
 onal)                                                  

### Use the model

In [14]:
# Labels ['fear', 'sadness', 'joy', 'anger', 'love', 'surprise']

In [15]:
def extract_emotion(text):
    sentence = preprocess_sentence(text)
    
    sentence = tokenizer.texts_to_sequences([sentence])
    sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
    
    result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))
    proba =  np.max(model.predict(sentence))

    # class_predict =  np.max(model.predict(sentence))
    # print(class_predict)
    
    print(f"{result} : {proba}\n\n")
    
    return result[0]

extract_emotion("Hurray!! My Model Got 93% Accuracy, Connect With me https://www.linkedin.com/in/asutoshpati/")

1/1 [==============================] - 0s 298ms/step
['fear'] : 0.32459312677383423




'fear'

In [16]:
example_sentences = [
    "He's over the moon about being accepted to the university",
    "Your point on this certain matter made me outrageous, how can you say so? This is insane.",
    "I can't do it, I'm not ready to lose anything, just leave me alone",
    "Merlin's beard harry, you can cast the Patronus charm! I'm amazed!",
    "I am amazed, that you do it."
]

In [17]:
for sentence in example_sentences:
    print(sentence)
    print(extract_emotion(sentence))
    print("\n\n")

He's over the moon about being accepted to the university
1/1 [==============================] - 0s 299ms/step
['joy'] : 0.8274199962615967


joy



Your point on this certain matter made me outrageous, how can you say so? This is insane.
1/1 [==============================] - 0s 483ms/step
['anger'] : 0.48978257179260254


anger



I can't do it, I'm not ready to lose anything, just leave me alone
1/1 [==============================] - 0s 323ms/step
['fear'] : 0.3876217007637024


fear



Merlin's beard harry, you can cast the Patronus charm! I'm amazed!
1/1 [==============================] - 0s 306ms/step
['surprise'] : 0.9352065324783325


surprise



I am amazed, that you do it.
1/1 [==============================] - 0s 298ms/step
['surprise'] : 0.2998327612876892


surprise





In [18]:
emotion_mood_mapping = {"fear": "F", "sadness": "S", "joy": "H", "anger": "A", "love": "L", "surprise": "N"}

In [19]:
import requests

def send_mood(mood):
    url = 'http://192.168.1.1'

    try:
        payload = {'mood': mood}
        response = requests.get(url, params=payload)

        if response.status_code == 200:
            print("Request sent to bot")
            return None
        else:
            print('Error:', response.status_code)
            return None
    except requests.exceptions.RequestException as e:
        print('Error:', e)
        return None

In [20]:
# Few examples that you can try
# Hey, I have a good news for you. are you amazed! - surprise
# Its your birthday, I have gift for you. aren't you happy. - joy
# What you've done has left me heartbroken. - sadness
# Are you afraid that I know what you have done ? - fear
# For your nasty work I am going to scold you - anger

In [21]:
# Don't forget to connect with CAP10 before running this cell

text = input("Enter your text: ")
emotion = extract_emotion(text)
mood = emotion_mood_mapping.get(emotion)
send_mood(mood)

Enter your text:  Hey, I have a good news for you. are you amazed!


1/1 [==============================] - 0s 266ms/step
['surprise'] : 0.6928766369819641


Error: HTTPConnectionPool(host='192.168.1.1', port=80): Max retries exceeded with url: /?mood=N (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000210F0931C10>, 'Connection to 192.168.1.1 timed out. (connect timeout=None)'))
